<h3>VVI</h3>

In [1]:
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.document_loaders import WebBaseLoader
from langchain.vectorstores import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain import hub
from langchain.agents import create_tool_calling_agent,AgentExecutor
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,SerpAPIWrapper
from langchain_core.tools import Tool

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
load_dotenv(find_dotenv("../.env"))

True

In [3]:
llmGemini=ChatGoogleGenerativeAI(model="gemini-1.5-flash")
llmOpenAI=ChatOpenAI(model="gpt-3.5-turbo")

<h3>Tavily Search Tool</h3>

In [4]:
searchTavily=TavilySearchResults(max_results=4)

In [5]:
searchTavily.invoke(input="How is the weather in Kolkata?")

[{'title': 'Weather in Kolkata in January 2026 - Detailed Forecast',
  'url': 'https://www.easeweather.com/asia/india/west-bengal/kolkata/january',
  'content': '| 17 Jan. | Sunny | 28° /14° | 0 mm | 6 |\n| 18 Jan. | Sunny | 29° /15° | 0 mm | 6 |\n| 19 Jan. | Sunny | 30° /16° | 0 mm | 6 |\n| 20 Jan. | Sunny | 31° /18° | 0 mm | 6 |\n| 21 Jan. | Sunny | 33° /19° | 0 mm | 7 |\n| 22 Jan. | Partly cloudy | 32° /20° | 0 mm | 7 |\n| 23 Jan. | Sunny | 31° /19° | 0 mm | 6 |\n| 24 Jan. | Sunny | 30° /17° | 0 mm | 6 |\n| 25 Jan. | Sunny | 30° /16° | 0 mm | 6 |\n| 26 Jan. | Sunny | 29° /15° | 0 mm | 6 |\n| 27 Jan. | Overcast | 27° /15° | 0 mm | 7 |\n| 28 Jan. | Sunny | 27° /14° | 0 mm | 7 |\n| 29 Jan. | Sunny | 28° /15° | 0 mm | 7 |\n| 30 Jan. | Sunny | 30° /17° | 0 mm | 7 |\n| 31 Jan. | Patchy rain possible | 30° /19° | 0.2 mm | 7 |\n| Next | [...] ### Temperatures\n\n Until now, January 2026 in Kolkata is slightly cooler than the historical average by -1.5 °C.\n The forecast for the upcoming day

<h3>Wikipedia Search</h3>

In [6]:
wikipediaSearch=WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

<h3>SerpAPI Search</h3>

In [7]:
serpAPISearch=Tool(
    func=SerpAPIWrapper(search_engine="bing").run,
    description="Search Tool from SerpAPI",
    name="serpAPI_Search")

<h3>Retrievers</h3>

In [21]:
# Loaders
loader=WebBaseLoader(web_path="https://timesofindia.indiatimes.com/india/kolkata-shocker-victim-alleges-rape-was-videotaped-what-initial-probe-revealed/articleshow/122113479.cms")

In [22]:
docs=loader.load()

In [23]:
docs

[Document(metadata={'source': 'https://timesofindia.indiatimes.com/india/kolkata-shocker-victim-alleges-rape-was-videotaped-what-initial-probe-revealed/articleshow/122113479.cms', 'title': 'Kolkata shocker: Victim alleges rape was videotaped; what initial probe revealed | India News - Times of India', 'description': 'India News: Kolkata Police reported the arrest of Manujit Mishra, a former Trinamool Congress Chhatra Parishad leader, for allegedly raping a law student within c', 'language': 'en'}, page_content='Kolkata shocker: Victim alleges rape was videotaped; what initial probe revealed | India News - Times of IndiaEditionININUSEnglishEnglishहिन्दीमराठीಕನ್ನಡதமிழ்বাংলাമലയാളംతెలుగుગુજરાતીSign InTOIindiaOperation SindoorCancel PatriarchyMaharashtraDelhiKarnatakaTamil NaduTelanganaUttar PradeshWest BengalGujaratMadhya PradeshBiharChandigarhRajasthanArunachal PradeshAndhra PradeshAssamChhattisgarhGoaHaryanaHimachal PradeshJammu KashmirJharkhandKeralaManipurMeghalayaMizoramNagalandOdisha

In [25]:
# Split Documents
documentsObj=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

In [26]:
documents=documentsObj.split_documents(documents=docs)

In [28]:
documents

[Document(metadata={'source': 'https://timesofindia.indiatimes.com/india/kolkata-shocker-victim-alleges-rape-was-videotaped-what-initial-probe-revealed/articleshow/122113479.cms', 'title': 'Kolkata shocker: Victim alleges rape was videotaped; what initial probe revealed | India News - Times of India', 'description': 'India News: Kolkata Police reported the arrest of Manujit Mishra, a former Trinamool Congress Chhatra Parishad leader, for allegedly raping a law student within c', 'language': 'en'}, page_content="Kolkata shocker: Victim alleges rape was videotaped; what initial probe revealed | India News - Times of IndiaEditionININUSEnglishEnglishहिन्दीमराठीಕನ್ನಡதமிழ்বাংলাമലയാളംతెలుగుગુજરાતીSign InTOIindiaOperation SindoorCancel PatriarchyMaharashtraDelhiKarnatakaTamil NaduTelanganaUttar PradeshWest BengalGujaratMadhya PradeshBiharChandigarhRajasthanArunachal PradeshAndhra PradeshAssamChhattisgarhGoaHaryanaHimachal PradeshJammu KashmirJharkhandKeralaManipurMeghalayaMizoramNagalandOdisha

In [29]:
len(documents)

17

In [30]:
# Use Vector DB to Insert Documents
vector=FAISS.from_documents(documents=documents,embedding=OpenAIEmbeddings())

In [31]:
retriever=vector.as_retriever(k=4)

In [32]:
retriever.invoke(input="Why does Coffee Makers struggles?")

[Document(id='d3018dd0-21db-4c64-8471-4c9589c493ba', metadata={'source': 'https://timesofindia.indiatimes.com/india/kolkata-shocker-victim-alleges-rape-was-videotaped-what-initial-probe-revealed/articleshow/122113479.cms', 'title': 'Kolkata shocker: Victim alleges rape was videotaped; what initial probe revealed | India News - Times of India', 'description': 'India News: Kolkata Police reported the arrest of Manujit Mishra, a former Trinamool Congress Chhatra Parishad leader, for allegedly raping a law student within c', 'language': 'en'}, page_content='MobileGadgetsWork with UsColombiaHot on the WebHeart HealthLiver Disease SymptomsVitamin D DeficiencyKumar VishwasShefali Jariwala Death Live UpdateKannappa Movie ReviewLargest CrocodilesHair Growth OilsMaa Box Office CollectionShefali Jariwala Death CausesJapan VillageHeart HealthMaa Movie ReviewPanchayat 4 ReviewSalman KhanKapil SharmaParag TyagiOptical illusionCardiac Arrest SymptomsKannappa ReviewYukta MookheyBlood SugarKannappa Box

<h3>Create Retriever Tool</h3>

In [33]:
retrieverTool=create_retriever_tool(
    retriever=retriever,
    name="Kolkata_harrassment_accussed",
    description="Search for information about the Kolkata Harrassment incident. For any questions about the Kolkata Harrassment, you must use this tool!"
)

In [34]:
# Use Both the Tools - search and retrievertool to use in the downstream
tools=[
    retrieverTool,
    searchTavily,
    serpAPISearch,
    wikipediaSearch]

In [35]:
prompt=hub.pull(owner_repo_commit="hwchase17/openai-functions-agent")

In [36]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [37]:
# The agent is responsible for taking in input and deciding what actions to take
# The agent does not execute those actions, that is done by the agent executor

agent=create_tool_calling_agent(llm=llmOpenAI,tools=tools,prompt=prompt)

In [38]:
agentExecutor=AgentExecutor(tools=tools,agent=agent,verbose=True)

In [39]:
agentExecutor.invoke(input={"input":"hi!"})



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'hi!', 'output': 'Hello! How can I assist you today?'}

In [40]:
agentExecutor.invoke(input={"input":"Who is the Prime Accussed?"})



> Entering new AgentExecutor chain...

Invoking: `Kolkata_harrassment_accussed` with `{'query': 'Prime Accused Kolkata Harassment'}`


Kolkata shocker: Victim alleges rape was videotaped; what initial probe revealed | India News - Times of IndiaEditionININUSEnglishEnglishहिन्दीमराठीಕನ್ನಡதமிழ்বাংলাമലയാളംతెలుగుગુજરાતીSign InTOIindiaOperation SindoorCancel PatriarchyMaharashtraDelhiKarnatakaTamil NaduTelanganaUttar PradeshWest BengalGujaratMadhya PradeshBiharChandigarhRajasthanArunachal PradeshAndhra PradeshAssamChhattisgarhGoaHaryanaHimachal PradeshJammu KashmirJharkhandKeralaManipurMeghalayaMizoramNagalandOdishaPunjabSikkimTripuraUttarakhandAndaman Nicobar IslandsDadra Nagar HaveliDaman DiuLakshadweepPondicherryWeb StoriesSaving Our StripesDigital ArrestToday's ePaperNewsIndia NewsKolkata shocker: Victim alleges rape was videotaped; what initial probe revealedTrendingCJI Bhushan GavaiWaziristan attackMann ki BaatNew Zealand VisaAnish GiriKolkata Gape Rape CaseIndian Origin CEOCBSE Sup

{'input': 'Who is the Prime Accussed?',
 'output': 'The prime accused in the Kolkata harassment incident is Manojit Mishra, who is a former Trinamool Congress Chhatra Parishad leader. He was arrested for allegedly raping a law student within college premises. Mishra was also a temporary staff member and a practicing criminal lawyer.'}

In [41]:
agentExecutor.invoke(input={'input':"What is the weather at Kolkata now?"})



> Entering new AgentExecutor chain...

Invoking: `serpAPI_Search` with `weather in Kolkata`


{'type': 'weather_result', 'temperature': '77', 'unit': 'Fahrenheit', 'precipitation': '0%', 'humidity': '43%', 'wind': '5 mph', 'location': 'Kolkata, West Bengal, India', 'date': 'Sunday 2:00 PM', 'weather': 'Sunny'}The weather in Kolkata right now is sunny with a temperature of 77°F. The humidity is at 43%, wind speed is 5 mph, and there is no precipitation.

> Finished chain.


{'input': 'What is the weather at Kolkata now?',
 'output': 'The weather in Kolkata right now is sunny with a temperature of 77°F. The humidity is at 43%, wind speed is 5 mph, and there is no precipitation.'}

<h3> Adding in Memory</h3>

In [42]:
messageHistory=ChatMessageHistory()

In [43]:
agentWithChatHistory=RunnableWithMessageHistory(
    runnable=agentExecutor,
    get_session_history=lambda session_id: messageHistory,
    input_message_key="input",
    history_messages_key="chat_history"
)

In [44]:
agentWithChatHistory.invoke(
    input={"input":"hi! My Name is Ritish"},
    config={"configurable":{"session_id":"<ritish>"}}
)



> Entering new AgentExecutor chain...
Hello Ritish! How can I assist you today?

> Finished chain.


{'input': 'hi! My Name is Ritish',
 'chat_history': [],
 'output': 'Hello Ritish! How can I assist you today?'}

In [45]:
agentWithChatHistory.invoke(
    input={"input":"What's my name?"},
    config={"configurable":{"session_id":"<ritish>"}}
)



> Entering new AgentExecutor chain...
Your name is Ritish. How can I help you further, Ritish?

> Finished chain.


{'input': "What's my name?",
 'chat_history': [HumanMessage(content='hi! My Name is Ritish', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello Ritish! How can I assist you today?', additional_kwargs={}, response_metadata={})],
 'output': 'Your name is Ritish. How can I help you further, Ritish?'}

In [46]:
agentWithChatHistory.invoke(
    input={"input":"What is the name of the victim?"},
    config={"configurable":{"session_id":"<ritish>"}}
)



> Entering new AgentExecutor chain...

Invoking: `Kolkata_harrassment_accussed` with `{'query': 'name of the victim'}`


Kolkata shocker: Victim alleges rape was videotaped; what initial probe revealed | India News - Times of IndiaEditionININUSEnglishEnglishहिन्दीमराठीಕನ್ನಡதமிழ்বাংলাമലയാളംతెలుగుગુજરાતીSign InTOIindiaOperation SindoorCancel PatriarchyMaharashtraDelhiKarnatakaTamil NaduTelanganaUttar PradeshWest BengalGujaratMadhya PradeshBiharChandigarhRajasthanArunachal PradeshAndhra PradeshAssamChhattisgarhGoaHaryanaHimachal PradeshJammu KashmirJharkhandKeralaManipurMeghalayaMizoramNagalandOdishaPunjabSikkimTripuraUttarakhandAndaman Nicobar IslandsDadra Nagar HaveliDaman DiuLakshadweepPondicherryWeb StoriesSaving Our StripesDigital ArrestToday's ePaperNewsIndia NewsKolkata shocker: Victim alleges rape was videotaped; what initial probe revealedTrendingCJI Bhushan GavaiWaziristan attackMann ki BaatNew Zealand VisaAnish GiriKolkata Gape Rape CaseIndian Origin CEOCBSE Supplementary Tim

{'input': 'What is the name of the victim?',
 'chat_history': [HumanMessage(content='hi! My Name is Ritish', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello Ritish! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Your name is Ritish. How can I help you further, Ritish?', additional_kwargs={}, response_metadata={})],
 'output': 'The victim in the Kolkata harassment incident has alleged that she was raped and filmed while being gang-raped. However, the specific name of the victim is not mentioned in the information available. If you need more details or have any other questions, feel free to ask.'}

In [47]:
agentWithChatHistory.invoke(
    input={"input":"Who won in the Iran-Israel war?"},
    config={"configurable":{"session_id":"<ritish>"}}
)



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Iran-Israel war'}`


Page: Iran–Israel war
Summary: The Iran–Israel war (13 – 24 June 2025), also known as the Twelve-Day War, was an armed conflict in the Middle East. The war began when Israel bombed military and nuclear facilities in Iran in a surprise attack, assassinating prominent military leaders, nuclear scientists, and politicians, killing civilians, and damaging or destroying air defenses. Iran retaliated with over 550 ballistic missiles and over 1,000 suicide drones, hitting civilian population centers, one hospital and at least twelve military, energy, and government sites. The United States intercepted Iranian attacks, and bombed three Iranian nuclear sites on 22 June. Iran retaliated by firing missiles at a US base in Qatar. On 24 June, Israel and Iran agreed to a ceasefire under US pressure.
The war followed a decades-long proxy war. Iran has called for Israel's destruction, and armed Hamas i

{'input': 'Who won in the Iran-Israel war?',
 'chat_history': [HumanMessage(content='hi! My Name is Ritish', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello Ritish! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Your name is Ritish. How can I help you further, Ritish?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is the name of the victim?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The victim in the Kolkata harassment incident has alleged that she was raped and filmed while being gang-raped. However, the specific name of the victim is not mentioned in the information available. If you need more details or have any other questions, feel free to ask.', additional_kwargs={}, response_metadata={})],
 'output': 'The Iran-Israel war, also known as the Twelve-Day War, took place fr

In [48]:
agentWithChatHistory.invoke(
    input={"input":"Which party won the election in Jammu and Kashmir?"},
    config={"configurable":{"session_id":"<ritish>"}}
)



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Jammu and Kashmir election 2022'}`


Page: Jammu and Kashmir Legislative Assembly
Summary: The Jammu and Kashmir Legislative Assembly (Kashmiri: جۆم تہٕ کٔشیٖر مَجلِسہِ قۄنوٗن سٲزی (in Perso-Arabic); 𑆘𑆾𑇌𑆩𑇀 𑆠 𑆑𑆯𑆵𑆫𑇀 𑆩𑆘𑇀𑆬𑆴𑆱𑇀-𑆅 𑆑𑇊𑆾𑇋𑆳𑆤𑆷𑆤𑇀 𑆱𑆃𑆘𑇊𑆵 (in Sharada)) is the unicameral legislature of Indian union territory of Jammu and Kashmir. Jammu and Kashmir Legislative Assembly is the legislative arm of the Government of Jammu and Kashmir. At present, it consists of 95 members, directly elected from 95 constituencies. The tenure of the Legislative Assembly is five years unless dissolved sooner.
Prior to 2019, the State of Jammu and Kashmir had a bicameral legislature with a legislative assembly (lower house) and a legislative council (upper house). The Jammu and Kashmir Reorganisation Act, passed by the Parliament of India in August 2019, replaced this with a unicameral legislature while also re-organising the state 

{'input': 'Which party won the election in Jammu and Kashmir?',
 'chat_history': [HumanMessage(content='hi! My Name is Ritish', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello Ritish! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Your name is Ritish. How can I help you further, Ritish?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is the name of the victim?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The victim in the Kolkata harassment incident has alleged that she was raped and filmed while being gang-raped. However, the specific name of the victim is not mentioned in the information available. If you need more details or have any other questions, feel free to ask.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Who won in the Iran-Israel war?', addi

In [49]:
agentWithChatHistory.invoke(
    input={"input":"What's are the questions that I have asked so far?"},
    config={"configurable":{"session_id":"<ritish>"}}
)



> Entering new AgentExecutor chain...
So far, you have asked the following questions:
1. What's my name?
2. What is the name of the victim?
3. Who won in the Iran-Israel war?
4. Which party won the election in Jammu and Kashmir?

If you have any more questions or need assistance with anything else, feel free to let me know!

> Finished chain.


{'input': "What's are the questions that I have asked so far?",
 'chat_history': [HumanMessage(content='hi! My Name is Ritish', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello Ritish! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Your name is Ritish. How can I help you further, Ritish?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is the name of the victim?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The victim in the Kolkata harassment incident has alleged that she was raped and filmed while being gang-raped. However, the specific name of the victim is not mentioned in the information available. If you need more details or have any other questions, feel free to ask.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Who won in the Iran-Israel war?', addi

In [50]:
response=agentWithChatHistory.invoke(
    input={"input":"What's my name?"},
    config={"configurable":{"session_id":"<ritish>"}}
)



> Entering new AgentExecutor chain...
Your name is Ritish. How can I assist you further, Ritish?

> Finished chain.


In [92]:
response

{'input': "What's my name?",
 'chat_history': [HumanMessage(content='hi! My Name is Ritish', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello Ritish! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Your name is Ritish. How can I assist you today, Ritish?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is the name of the victim?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The victim in the Kolkata Harrassment incident is a law student who alleged that she was raped by the accused, who also filmed the assault and threatened to release it. The prime accused is Manujit Mishra, a former Trinamool Congress Chhatra Parishad leader.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Who won in the Iran-Israel war?', additional_kwargs={}, response_metadata={}),
  AIMes